In [82]:
import tensorflow as tf
import os
import numpy as np
# from copy import deepcopy

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.optimizers import Adam
from keras import backend as K
import functools
from collections import Counter

In [83]:
# load data from CIFAR10
def load_data(clean_data_size):
    cifar10 = tf.keras.datasets.cifar10
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    x_train = x_train.reshape(x_train.shape[0], 32, 32, 3)
    x_test = x_test.reshape(x_test.shape[0], 32, 32, 3)

    # transform labels to one-hot vectors
    y_train = tf.contrib.keras.utils.to_categorical(y_train, 10)
    y_test = tf.contrib.keras.utils.to_categorical(y_test, 10)
    
    clean_index = []
    for label in range(10):
        positive_index = list(np.where(y_train[:, label] == 1)[0])        
        clean_index = np.append(clean_index, np.random.choice(positive_index, clean_data_size, replace=False)).astype(int)
    
    x_clean = x_train[clean_index]
    y_clean = y_train[clean_index]
    x_train = np.delete(x_train, clean_index, axis=0)
    y_train = np.delete(y_train, clean_index, axis=0)
    return x_train, y_train, x_test, y_test, x_clean, y_clean


def generate_noise_labels(y_train, noise_level):
    num_noise = int(noise_level * y_train.shape[0])
    noise_index = np.random.choice(y_train.shape[0], num_noise, replace=False)
    label_slice = np.argmax(y_train[noise_index], axis=1)
    new_label = np.random.randint(low=0, high=10, size=num_noise)
    while sum(label_slice == new_label) > 0:
        n = sum(label_slice == new_label)
        new_label[label_slice == new_label] = np.random.randint(low=0, high=10, size=n)
    y_train[noise_index] = tf.contrib.keras.utils.to_categorical(new_label, 10)
    return y_train


In [84]:
noise_level = 0.8
clean_data_size = 50

x_train, y_train, x_test, y_test, x_clean, y_clean = load_data(clean_data_size)
y_train = generate_noise_labels(y_train, noise_level)


In [85]:
def create_model(architecture, num_classes, learning_rate=0.0002, dropout=0.5):
    model = Sequential()
    for layer_index in range(len(architecture)):
        layer = architecture[layer_index]
        if len(layer) == 3:
            if layer_index == 0:
                model.add(Conv2D(layer[0], kernel_size=(layer[1], layer[2]), input_shape=(32, 32, 3),
                                 kernel_initializer='glorot_normal', activation='relu', padding='same'))
            else:
                model.add(Conv2D(layer[0], kernel_size=(layer[1], layer[2]), kernel_initializer='glorot_normal',
                                 activation='relu', padding='same'))
            if layer_index < 3:
                model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        elif len(layer) == 1:
            if len(architecture[layer_index - 1]) == 3:
                model.add(Flatten())
            model.add(Dense(layer[0], activation='relu', kernel_initializer='glorot_normal'))
        else:
            print('Invalid architecture /(ㄒoㄒ)/~~')
    model.add(Dropout(dropout))
    if num_classes > 2:
        model.add(Dense(num_classes))
        model.add(Activation('softmax'))
        adam = Adam(lr=learning_rate)
        model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=adam)
    elif num_classes == 2:
        model.add(Dense(1))
        model.add(Activation('sigmoid'))
        adam = Adam(lr=learning_rate)
        model.compile(loss='mean_squared_error', metrics=['accuracy'], optimizer=adam)   
    return model

In [86]:
# generate 10 binary classifier
binary_classifier_list = []
architecture = [[32, 5, 5], [32, 5, 5], [32, 5, 5], [256]]
for label in range(10):
    model = create_model(architecture, num_classes=2)
    binary_classifier_list.append(model)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_58 (Conv2D)           (None, 32, 32, 32)        2432      
_________________________________________________________________
max_pooling2d_58 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_59 (Conv2D)           (None, 16, 16, 32)        25632     
_________________________________________________________________
max_pooling2d_59 (MaxPooling (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_60 (Conv2D)           (None, 8, 8, 32)          25632     
_________________________________________________________________
max_pooling2d_60 (MaxPooling (None, 4, 4, 32)          0         
_________________________________________________________________
flatten_20 (Flatten)         (None, 512)               0         
__________

In [87]:
# use the idea of PU learning to augment positive data
epochs = 20
additional_data_index = [[] for i in range(10)]
for epoch in range(epochs):
    for label in range(10):
        positive_index = list(np.where(y_clean[:, label] == 1)[0])
        x = x_clean[positive_index]
        x = np.concatenate((x, x_train[additional_data_index[label]]), axis=0)
        n_p = len(x)
        n_n = min(400, n_p)
        negative_index = list(np.where(y_clean[:, label] != 1)[0])
        negative_index = np.random.choice(negative_index, n_n, replace=False)
        x = np.concatenate((x, x_clean[negative_index]), axis=0)
        y = [1] * n_p + [0] * n_n
        classifier = binary_classifier_list[label]
        classifier.fit(x, y, batch_size=32, epochs=20, shuffle=True)
        pred_train = classifier.predict(x_train)
        candidate_index = np.where(pred_train > 0.98)[0]
        if len(candidate_index) < 1000:
            additional_data_index[label] = list(candidate_index)
        else:
            additional_data_index[label] = np.argsort(-pred_train, axis=0)[0:1000].reshape(-1)

Epoch 1/20
100/100 [==============================] - 1s 13ms/step - loss: 0.2590 - acc: 0.4100
Epoch 2/20
100/100 [==============================] - 0s 2ms/step - loss: 0.2549 - acc: 0.4500
Epoch 3/20
100/100 [==============================] - 0s 1ms/step - loss: 0.2476 - acc: 0.4800
Epoch 4/20
100/100 [==============================] - 0s 1ms/step - loss: 0.2470 - acc: 0.4900
Epoch 5/20
100/100 [==============================] - 0s 1ms/step - loss: 0.2439 - acc: 0.5500
Epoch 6/20
100/100 [==============================] - 0s 1ms/step - loss: 0.2432 - acc: 0.6200
Epoch 7/20
100/100 [==============================] - 0s 1ms/step - loss: 0.2386 - acc: 0.6700
Epoch 8/20
100/100 [==============================] - 0s 1ms/step - loss: 0.2298 - acc: 0.7500
Epoch 9/20
100/100 [==============================] - 0s 1ms/step - loss: 0.2265 - acc: 0.6900
Epoch 10/20
100/100 [==============================] - 0s 1ms/step - loss: 0.2239 - acc: 0.7600
Epoch 11/20
100/100 [===========================

100/100 [==============================] - 0s 817us/step - loss: 0.2466 - acc: 0.5500
Epoch 6/20
100/100 [==============================] - 0s 926us/step - loss: 0.2541 - acc: 0.5000
Epoch 7/20
100/100 [==============================] - 0s 827us/step - loss: 0.2465 - acc: 0.5100
Epoch 8/20
100/100 [==============================] - 0s 1ms/step - loss: 0.2389 - acc: 0.5600
Epoch 9/20
100/100 [==============================] - 0s 794us/step - loss: 0.2442 - acc: 0.5200
Epoch 10/20
100/100 [==============================] - 0s 913us/step - loss: 0.2404 - acc: 0.5500
Epoch 11/20
100/100 [==============================] - 0s 952us/step - loss: 0.2381 - acc: 0.5400
Epoch 12/20
100/100 [==============================] - 0s 1ms/step - loss: 0.2365 - acc: 0.5100
Epoch 13/20
100/100 [==============================] - 0s 1ms/step - loss: 0.2432 - acc: 0.5200
Epoch 14/20
100/100 [==============================] - 0s 895us/step - loss: 0.2323 - acc: 0.5600
Epoch 15/20
100/100 [=====================

100/100 [==============================] - 0s 768us/step - loss: 0.1590 - acc: 0.7700
Epoch 10/20
100/100 [==============================] - 0s 885us/step - loss: 0.1574 - acc: 0.8100
Epoch 11/20
100/100 [==============================] - 0s 905us/step - loss: 0.1568 - acc: 0.7800
Epoch 12/20
100/100 [==============================] - 0s 775us/step - loss: 0.1524 - acc: 0.7800
Epoch 13/20
100/100 [==============================] - 0s 904us/step - loss: 0.1478 - acc: 0.7800
Epoch 14/20
100/100 [==============================] - 0s 1ms/step - loss: 0.1438 - acc: 0.8100
Epoch 15/20
100/100 [==============================] - 0s 1ms/step - loss: 0.1510 - acc: 0.8100
Epoch 16/20
100/100 [==============================] - 0s 821us/step - loss: 0.1391 - acc: 0.8000
Epoch 17/20
100/100 [==============================] - 0s 1ms/step - loss: 0.1491 - acc: 0.7800
Epoch 18/20
100/100 [==============================] - 0s 896us/step - loss: 0.1782 - acc: 0.7500
Epoch 19/20
100/100 [=================

100/100 [==============================] - 0s 954us/step - loss: 0.1255 - acc: 0.8500
Epoch 14/20
100/100 [==============================] - 0s 1ms/step - loss: 0.1195 - acc: 0.8600
Epoch 15/20
100/100 [==============================] - 0s 1ms/step - loss: 0.1133 - acc: 0.8600
Epoch 16/20
100/100 [==============================] - 0s 895us/step - loss: 0.1087 - acc: 0.8500
Epoch 17/20
100/100 [==============================] - 0s 887us/step - loss: 0.1235 - acc: 0.8600
Epoch 18/20
100/100 [==============================] - 0s 931us/step - loss: 0.1045 - acc: 0.8600
Epoch 19/20
100/100 [==============================] - 0s 783us/step - loss: 0.1095 - acc: 0.8500
Epoch 20/20
100/100 [==============================] - 0s 895us/step - loss: 0.1074 - acc: 0.8700
Epoch 1/20
100/100 [==============================] - 0s 802us/step - loss: 0.1689 - acc: 0.7600
Epoch 2/20
100/100 [==============================] - 0s 944us/step - loss: 0.1768 - acc: 0.7300
Epoch 3/20
100/100 [==================

100/100 [==============================] - 0s 800us/step - loss: 0.1341 - acc: 0.8400
Epoch 18/20
100/100 [==============================] - 0s 782us/step - loss: 0.1642 - acc: 0.7100
Epoch 19/20
100/100 [==============================] - 0s 791us/step - loss: 0.1631 - acc: 0.8100
Epoch 20/20
100/100 [==============================] - 0s 808us/step - loss: 0.1409 - acc: 0.8300
Epoch 1/20
122/122 [==============================] - 0s 816us/step - loss: 0.1280 - acc: 0.8443
Epoch 2/20
122/122 [==============================] - 0s 795us/step - loss: 0.1293 - acc: 0.8279
Epoch 3/20
122/122 [==============================] - 0s 790us/step - loss: 0.1231 - acc: 0.8361
Epoch 4/20
122/122 [==============================] - 0s 834us/step - loss: 0.1137 - acc: 0.8443
Epoch 5/20
122/122 [==============================] - 0s 808us/step - loss: 0.1115 - acc: 0.8443
Epoch 6/20
122/122 [==============================] - 0s 755us/step - loss: 0.1191 - acc: 0.8197
Epoch 7/20
122/122 [==================

...

1450/1450 [==============================] - 2s 1ms/step - loss: 5.1870e-07 - acc: 1.0000
Epoch 4/20
1450/1450 [==============================] - 2s 1ms/step - loss: 1.0681e-06 - acc: 1.0000
Epoch 5/20
1450/1450 [==============================] - 1s 886us/step - loss: 6.9541e-07 - acc: 1.0000
Epoch 6/20
1450/1450 [==============================] - 1s 758us/step - loss: 8.1316e-07 - acc: 1.0000
Epoch 7/20
1450/1450 [==============================] - 1s 740us/step - loss: 1.4011e-06 - acc: 1.0000
Epoch 8/20
1450/1450 [==============================] - 1s 726us/step - loss: 7.7928e-07 - acc: 1.0000
Epoch 9/20
1450/1450 [==============================] - 2s 1ms/step - loss: 5.2326e-07 - acc: 1.0000
Epoch 10/20
1450/1450 [==============================] - 2s 1ms/step - loss: 1.0742e-06 - acc: 1.0000
Epoch 11/20
1450/1450 [==============================] - 2s 1ms/step - loss: 6.8338e-07 - acc: 1.0000
Epoch 12/20
1450/1450 [==============================] - 1s 1ms/step - loss: 9.5604e-07 - ac

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1450/1450 [==============================] - 2s 1ms/step - loss: 4.6431e-06 - acc: 1.0000
Epoch 18/20
1450/1450 [==============================] - 2s 1ms/step - loss: 7.0000e-06 - acc: 1.0000
Epoch 19/20
1450/1450 [==============================] - 2s 1ms/step - loss: 5.1483e-06 - acc: 1.0000
Epoch 20/20
1450/1450 [==============================] - 1s 712us/step - loss: 2.8693e-06 - acc: 1.0000
Epoch 1/20
1450/1450 [==============================] - 1s 743us/step - loss: 8.8868e-07 - acc: 1.0000
Epoch 2/20
1450/1450 [==============================] - 2s 1ms/step - loss: 6.3073e-07 - acc: 1.0000
Epoch 3/20
1450/1450 [==============================] - 2s 1ms/step - loss: 2.8316e-07 - acc: 1.0000
Epoch 4/20
1450/1450 [==============================] - 2s 1ms/step - loss: 9.8013e-07 - acc: 1.0000
Epoch 5/20
1450/1450 [==============================] - 1s 733us/step - loss: 5.4695e-07 - acc: 1.0000
Epoch 6/20
1450/1450 [==============================] - 1s 758us/step - loss: 9.4394e-07 - ac

1450/1450 [==============================] - 2s 1ms/step - loss: 6.9086e-04 - acc: 0.9993
Epoch 18/20
1450/1450 [==============================] - 2s 1ms/step - loss: 7.1905e-04 - acc: 0.9993
Epoch 19/20
1450/1450 [==============================] - 2s 1ms/step - loss: 0.0325 - acc: 0.9586
Epoch 20/20
1450/1450 [==============================] - 1s 823us/step - loss: 0.0118 - acc: 0.9848
Epoch 1/20
1450/1450 [==============================] - 1s 756us/step - loss: 0.0014 - acc: 0.9986
Epoch 2/20
1450/1450 [==============================] - 2s 1ms/step - loss: 0.0014 - acc: 0.9986
Epoch 3/20
1450/1450 [==============================] - 2s 1ms/step - loss: 0.0014 - acc: 0.9986
Epoch 4/20
1450/1450 [==============================] - 2s 1ms/step - loss: 0.0014 - acc: 0.9986
Epoch 5/20
1450/1450 [==============================] - 2s 1ms/step - loss: 0.0014 - acc: 0.9986
Epoch 6/20
1450/1450 [==============================] - 1s 740us/step - loss: 0.0014 - acc: 0.9986
Epoch 7/20
1450/1450 [==

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
# get additional data and train teacher model
for label in range(10):
    index = additional_data_index[label]
    x_clean = np.concatenate((x_clean, x_train[index]), axis=0)
    y_clean = np.concatenate((y_clean, tf.contrib.keras.utils.to_categorical([label]*len(index), 10)))

architecture = [[32, 5, 5], [32, 5, 5], [32, 5, 5], [500]]
teacher_model = create_model(architecture, num_classes=10)
teacher_model.fit(x_clean, y_clean, batch_size=64, epochs=50, shuffle=True)

In [ ]:
# calculate lambda
def get_precision(x, y, model):
    mean_precision = 0
    prediction = model.predict(x)
    for label in range(10):        
        pred_positive_index = list(np.where(prediction[:, label] == 1)[0])
        true_positive_index = list(np.where(y[:, label] != 1)[0])
        TP = len(list(set(pred_positive_index) & set(true_positive_index)))
        mean_precision += TP/len(pred_positive_index)

    return mean_precision/10

precision_whole = get_precision(x_trian, y_trian, teacher_model)
precision_clean = get_precision(x_clean, y_clean, teacher_model)
lambda_teacher = precision_clean / (precision_clean + precision_whole)

In [ ]:
# generate a multi-classifier
architecture = [[32, 5, 5], [32, 5, 5], [32, 5, 5], [500]]
student_model = create_model(architecture, num_classes=10)


In [ ]:
# train